In [170]:
# Import Library
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from google.colab import drive

In [172]:
# Mount Google Drive
drive.mount('/content/drive')

# Path ke dataset di Google Drive
dataset_path = "/content/drive/MyDrive/dataset/Food_Recipe.csv"

# Baca dataset menggunakan pandas
df = pd.read_csv(dataset_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [173]:
# Menampilkan 20 baris pertama dari DataFrame 'df'.
display(df.head(20))

# Menampilkan jumlah baris dan kolom dalam dataset
num_rows, num_cols = df.shape
print(f"Jumlah baris: {num_rows}")
print(f"Jumlah kolom: {num_cols}")

,name,description,cuisine,course,diet,ingredients_name,ingredients_quantity,prep_time (in mins),cook_time (in mins),instructions,image_url
0,Khandeshi Dal Recipe,Khandeshi Dal is definitely a new name for alm...,Maharashtrian Recipes,Lunch,High Protein Vegetarian,"Green Moong Dal (Whole), Pink Masoor Dal (Spli...",2 tablespoons Green Moong Dal (Whole) 2 table...,10.0,30.0,"To begin making the Khandeshi Dal recipe, we w...",https://www.archanaskitchen.com/images/archana...
1,Padipe Saasmi Recipe - Udupi Style Amaranth Le...,Cuisine of our country is amazing and the vari...,Udupi,Side Dish,Vegetarian,"Red Amaranth Leaves, Fresh coconut, Dry Red Ch...","12 sprig Red Amaranth Leaves , You can use spi...",15.0,20.0,"To begin making the Padipe Saasmi Recipe, pluc...",https://www.archanaskitchen.com/images/archana...
2,Creamy Polenta Recipe - Savory Cornmeal Porridge,Creamy Polenta Recipe is a a dish common in It...,Italian Recipes,Side Dish,Vegetarian,"Polenta, Water, Milk, Extra Virgin Olive Oil, ...",1 cup Polenta 4 cups Water 1 cup Milk 1 tab...,0.0,20.0,"To begin making the Creamy Polenta Recipe, hea...",https://www.archanaskitchen.com/images/archana...
3,Summer Lettuce Salad Recipe,The Summer Lettuce Salad Recipe is perfect for...,Continental,Appetizer,Vegan,"Cherry tomatoes, Red onion, Cucumber, Sweet co...","10 Cherry tomatoes , halved 1 Red onion , ong...",20.0,0.0,"To begin making Summer Lettuce Salad Recipe, c...",https://www.archanaskitchen.com/images/archana...
4,Dondakkai Puli Curry Recipe - South Indian Sty...,Dondakkai Puli Curry is one of those tradition...,Tamil Nadu,Lunch,Vegetarian,"Tindora (Dondakaya/ Kovakkai), Black Urad Dal ...",150 grams Tindora (Dondakaya/ Kovakkai) 3 tab...,20.0,25.0,To begin making the Dondakkai Puli Curry Recip...,https://www.archanaskitchen.com/images/archana...
5,Kasuri Methi Aur Aloo Rasedar Recipe,Kasuri Methi Aur Aloo Rasedar Recipe is an eas...,North Indian Recipes,Side Dish,Vegetarian,"Potatoes (Aloo), Tomatoes, Ginger, Green Chill...","2 cups Potatoes (Aloo) , boiled and roughly ma...",20.0,10.0,To begin making Kasuri Methi Aur Aloo Rasedar ...,https://www.archanaskitchen.com/images/archana...
6,Poosanikai Haddu Sweet Curry Recipe - Pumpkin ...,Pumpkin Sweet Curry Recipe is a South Indian V...,Tamil Nadu,Side Dish,Vegetarian,"Kaddu (Parangikai/ Pumpkin), Mustard seeds (Ra...","250 grams Kaddu (Parangikai/ Pumpkin) , diced ...",10.0,20.0,To begin making the Pumpkin Sweet Curry Recipe...,https://www.archanaskitchen.com/images/archana...
7,Konkani Style Vengaya Sagle Recipe - Eggplant ...,Konkani Style Vengaya Sagle Recipe is a very f...,Konkan,Lunch,Vegetarian,"Drumstick, Brinjal (Baingan / Eggplant), Musta...","1 Drumstick , cut into 2 inch long 4 Brinjal ...",10.0,25.0,To begin making the Konkani Style Vengaya Sagl...,https://www.archanaskitchen.com/images/archana...
8,Ayam Goreng Kremes Recipe - Indonesian Style C...,Ayam Goreng Kremes is chicken made Indonesian ...,Indonesian,Appetizer,High Protein Non Vegetarian,"Cashew nuts, Garlic, Onion, Lemon juice, Kaffi...",1/4 cup Cashew nuts 8 cloves Garlic 1 Onion ...,10.0,20.0,"To begin making the Ayam Goreng Kremes, in a m...",https://www.archanaskitchen.com/images/archana...
9,Mangalore Cucumber Chana Dal Subzi Recipe,Mangalore Cucumber Chana Dal Sabzi Recipe is a...,Mangalorean,Lunch,Vegetarian,"Mangalorean cucumber, Chana dal (Bengal Gram D...","1 Mangalorean cucumber , cut into bite sized p...",10.0,30.0,To begin making the Mangalore Cucumber Chana D...,https://www.archanaskitchen.com/images/archana...


Jumlah baris: 6911
Jumlah kolom: 11


In [174]:
# Menampilkan informasi mengenai jumlah baris dan kolom, tipe data dari setiap kolom, serta jumlah nilai non-null dalam setiap kolom.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6911 entries, 0 to 6910
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  6911 non-null   object 
 1   description           6910 non-null   object 
 2   cuisine               6853 non-null   object 
 3   course                6775 non-null   object 
 4   diet                  6779 non-null   object 
 5   ingredients_name      6904 non-null   object 
 6   ingredients_quantity  6904 non-null   object 
 7   prep_time (in mins)   6888 non-null   float64
 8   cook_time (in mins)   6888 non-null   float64
 9   instructions          6911 non-null   object 
 10  image_url             6911 non-null   object 
dtypes: float64(2), object(9)
memory usage: 594.0+ KB


In [175]:
# Menghitung dan menampilkan jumlah nilai yang hilang (null) untuk setiap kolom dalam DataFrame 'df'.
df.isna().sum()

name                      0
description               1
cuisine                  58
course                  136
diet                    132
ingredients_name          7
ingredients_quantity      7
prep_time (in mins)      23
cook_time (in mins)      23
instructions              0
image_url                 0
dtype: int64

In [177]:
# Daftar kolom yang akan dihapus
columns_to_drop = ['description', 'course', 'diet', 'prep_time (in mins)']

# Ubah nama kolom menjadi huruf kapital untuk huruf pertama dan huruf kecil untuk sisanya
df.columns = df.columns.str.capitalize()

# Menyusun urutan kolom sesuai dengan yang diinginkan
desired_columns = ['Name', 'Image_url', 'Cuisine', 'Cook_time (in mins)', 'Ingredients_name', 'Ingredients_quantity', 'Instructions']
df = df.reindex(columns=desired_columns)

# Menampilkan DataFrame setelah perubahan
print("\nDataFrame setelah perubahan:")
display(df.head())

# Menampilkan informasi dataset setelah diproses
print("\nJumlah baris dan kolom sekarang:")
print(df.shape)


DataFrame setelah perubahan:


,Name,Image_url,Cuisine,Cook_time (in mins),Ingredients_name,Ingredients_quantity,Instructions
0,Khandeshi Dal Recipe,https://www.archanaskitchen.com/images/archana...,Maharashtrian Recipes,30.0,"Green Moong Dal (Whole), Pink Masoor Dal (Spli...",2 tablespoons Green Moong Dal (Whole) 2 table...,"To begin making the Khandeshi Dal recipe, we w..."
1,Padipe Saasmi Recipe - Udupi Style Amaranth Le...,https://www.archanaskitchen.com/images/archana...,Udupi,20.0,"Red Amaranth Leaves, Fresh coconut, Dry Red Ch...","12 sprig Red Amaranth Leaves , You can use spi...","To begin making the Padipe Saasmi Recipe, pluc..."
2,Creamy Polenta Recipe - Savory Cornmeal Porridge,https://www.archanaskitchen.com/images/archana...,Italian Recipes,20.0,"Polenta, Water, Milk, Extra Virgin Olive Oil, ...",1 cup Polenta 4 cups Water 1 cup Milk 1 tab...,"To begin making the Creamy Polenta Recipe, hea..."
3,Summer Lettuce Salad Recipe,https://www.archanaskitchen.com/images/archana...,Continental,0.0,"Cherry tomatoes, Red onion, Cucumber, Sweet co...","10 Cherry tomatoes , halved 1 Red onion , ong...","To begin making Summer Lettuce Salad Recipe, c..."
4,Dondakkai Puli Curry Recipe - South Indian Sty...,https://www.archanaskitchen.com/images/archana...,Tamil Nadu,25.0,"Tindora (Dondakaya/ Kovakkai), Black Urad Dal ...",150 grams Tindora (Dondakaya/ Kovakkai) 3 tab...,To begin making the Dondakkai Puli Curry Recip...



Jumlah baris dan kolom sekarang:
(6911, 7)


In [178]:
# Fungsi untuk mendeteksi karakter bahasa India
def contains_indian_language(text):
    # Regex untuk mendeteksi karakter bahasa India (Unicode range \u0900-\u097F)
    indian_language_regex = re.compile(r'[\u0900-\u097F]+')
    if indian_language_regex.search(str(text)):
        return True
    return False

# Mendeteksi semua baris yang mengandung karakter bahasa India
rows_with_indian_language = df.applymap(contains_indian_language).any(axis=1)

# Mengambil semua baris yang mengandung bahasa India
indian_language_rows = df[rows_with_indian_language]

# Menampilkan semua baris yang mengandung bahasa India
display(indian_language_rows)

,Name,Image_url,Cuisine,Cook_time (in mins),Ingredients_name,Ingredients_quantity,Instructions
355,Green Moong Dal Sabzi (Recipe In Hindi),https://www.archanaskitchen.com/images/archana...,Gujarati Recipes﻿,45.0,"हरी मूंग दाल, जीरा, कढ़ी पत्ता, अदरक, हरी मिर्च...","1/2 कप हरी मूंग दाल , या हरे मूंग 1 छोटा चमच्...","To make green moong sabzi, first soak the moon..."
770,Cabbage Tomato Sabzi (Recipe In Hindi),https://www.archanaskitchen.com/images/archana...,North Indian Recipes,15.0,"पत्ता गोभी, टमाटर, हल्दी पाउडर, धनिया पाउडर, न...","300 ग्राम्स पत्ता गोभी , पतला काट ले 2 टमाटर ...","To make cabbage curry, first heat oil in a pre..."
774,Nuvvula Pachadi Recipe - Nuvvula Pachadi Recipe,https://www.archanaskitchen.com/images/archana...,Indian,10.0,"तिल (सफ़ेद), लाल मिर्च पाउडर, इमली का पानी, दही...",1 कप तिल (सफ़ेद) 1 छोटा चमच्च लाल मिर्च पाउडर ...,"To make Nuvvula Pachadi recipe, first heat a p..."
1326,Ragi Badam Soup Recipe,https://www.archanaskitchen.com/images/archana...,Indian,10.0,"रागी का आटा, प्याज, अदरक, गाजर, बादाम, हरा बीन...","2 बड़े चम्मच रागी का आटा 1 प्याज , बारीक काट ल...","To make Ragi Almond Soup recipe, first soak th..."
1624,चीज़ी आलू पराठा - Cheesy Aloo Paratha (Recipe I...,https://www.archanaskitchen.com/images/archana...,North Indian Recipes,15.0,"गेहूं का आटा, तेल, नमक, पानी, आलू, तेल, सुखी ल...",आटे के लिए 1 कप गेहूं का आटा 1 छोटा चमच्च तेल...,चीज़ी आलू पराठा बनाने के लिए सबसे पहले एक बड़ा ब...
...,...,...,...,...,...,...,...
6471,कश्मीरी दम मोंजी रेसिपी - Knol Khol in Yogurt ...,https://www.archanaskitchen.com/images/archana...,Kashmiri,30.0,"नोल नोल, सरसो का तेल, अदरक लहसुन का पेस्ट , कश...","3 नोल नोल , हरे पत्तो के साथ 4 बड़ा चमच्च सरस...",कश्मीरी दम मोंजी बनाने के लिए सबसे पहले नोल नो...
6482,अंकुरित दालो की सब्ज़ी रेसिपी - Mixed Sprouts S...,https://www.archanaskitchen.com/images/archana...,Indian,30.0,"अंकुरित दाल , पानी , तेल, जीरा, हींग, प्याज, अ...","2 कटोरी अंकुरित दाल पानी , प्रयोग अनुसार 2 ...",अंकुरित दालो की सब्ज़ी बनाने के लिए सबसे पहले ड...
6483,Peshawari Kala Chana Recipe – Peshawari Kala C...,https://www.archanaskitchen.com/images/archana...,Pakistani,30.0,"काला चना, प्याज, टमाटर, अदरक, हरी मिर्च, दालची...","2 cups black gram, sprouted 1 onion, finely ch...","To make Peshawari Kala Chana, first cook the b..."
6491,कश्मीरी वेठ चमन रेसिपी - Paneer in Yogurt Toma...,https://www.archanaskitchen.com/images/archana...,Kashmiri,20.0,"पनीर, पानी , हींग, दालचीनी, इलाईची, तेजपत्ता, ...","300 ग्राम्स पनीर , क्यूब्स में काटा हुआ 1-1/...",कश्मीरी वेठ चमन बनाने के लिए सबसे पहले कढ़ाई मे...


In [179]:
# Fungsi untuk menghapus karakter bahasa India
def remove_indian_language(text):
    indian_language_regex = re.compile(r'[\u0900-\u097F]+')
    cleaned_text = indian_language_regex.sub('', str(text))
    # Jika teks setelah pembersihan hanya spasi atau kosong, kembalikan None
    if cleaned_text.strip() == '':
        return None
    else:
        return cleaned_text

# Menerapkan fungsi penghapusan karakter bahasa India ke semua kolom
df = df.applymap(remove_indian_language)

# Menghapus baris yang mengandung kolom kosong
df_cleaned = df.dropna(how='any')

# Menampilkan beberapa baris pertama dari dataframe yang telah diproses
print("\nBeberapa baris pertama dari dataset yang telah dibersihkan:")
display(df_cleaned.head())

# Memeriksa apakah masih ada karakter bahasa India di dataframe yang telah dibersihkan
def contains_indian_language(text):
    indian_language_regex = re.compile(r'[\u0900-\u097F]+')
    return bool(indian_language_regex.search(str(text)))

contains_indian = df_cleaned.applymap(contains_indian_language).any().any()
if not contains_indian:
    print("\nTidak ada lagi karakter bahasa India di dalam dataset yang telah dibersihkan.")
else:
    print("\nMasih ada karakter bahasa India di dalam dataset yang telah dibersihkan.")


Beberapa baris pertama dari dataset yang telah dibersihkan:


,Name,Image_url,Cuisine,Cook_time (in mins),Ingredients_name,Ingredients_quantity,Instructions
0,Khandeshi Dal Recipe,https://www.archanaskitchen.com/images/archana...,Maharashtrian Recipes,30.0,"Green Moong Dal (Whole), Pink Masoor Dal (Spli...",2 tablespoons Green Moong Dal (Whole) 2 table...,"To begin making the Khandeshi Dal recipe, we w..."
1,Padipe Saasmi Recipe - Udupi Style Amaranth Le...,https://www.archanaskitchen.com/images/archana...,Udupi,20.0,"Red Amaranth Leaves, Fresh coconut, Dry Red Ch...","12 sprig Red Amaranth Leaves , You can use spi...","To begin making the Padipe Saasmi Recipe, pluc..."
2,Creamy Polenta Recipe - Savory Cornmeal Porridge,https://www.archanaskitchen.com/images/archana...,Italian Recipes,20.0,"Polenta, Water, Milk, Extra Virgin Olive Oil, ...",1 cup Polenta 4 cups Water 1 cup Milk 1 tab...,"To begin making the Creamy Polenta Recipe, hea..."
3,Summer Lettuce Salad Recipe,https://www.archanaskitchen.com/images/archana...,Continental,0.0,"Cherry tomatoes, Red onion, Cucumber, Sweet co...","10 Cherry tomatoes , halved 1 Red onion , ong...","To begin making Summer Lettuce Salad Recipe, c..."
4,Dondakkai Puli Curry Recipe - South Indian Sty...,https://www.archanaskitchen.com/images/archana...,Tamil Nadu,25.0,"Tindora (Dondakaya/ Kovakkai), Black Urad Dal ...",150 grams Tindora (Dondakaya/ Kovakkai) 3 tab...,To begin making the Dondakkai Puli Curry Recip...



Tidak ada lagi karakter bahasa India di dalam dataset yang telah dibersihkan.


In [180]:
# Menampilkan informasi dataset setelah diproses
print("\nJumlah baris dan kolom sekarang:")
print(df.shape)


Jumlah baris dan kolom sekarang:
(6911, 7)


Jumlah baris sebelum dan sesudah penghapusan baris kosong tetap sama karena tidak ada baris yang benar-benar kosong (NaN) setelah penghapusan karakter bahasa India, sehingga df.dropna() tidak menghapus baris apa pun.

In [181]:
# Menghapus nilai kosong (missing values)
df.dropna(inplace=True)

# Menghapus duplikat
df.drop_duplicates(inplace=True)

In [182]:
# Simpan dataset yang sudah di clean di Google Drive
file_path = '/content/drive/MyDrive/dataset/Food_Recipe_Cleaned.csv'
df.to_csv(file_path, index=False)

In [185]:
# Path ke dataset di Google Drive
dataset_path = "/content/drive/MyDrive/dataset/Food_Recipe_Cleaned.csv"

# Baca dataset menggunakan pandas
df = pd.read_csv(dataset_path)

In [186]:
# Menampilkan dataframe yang telah diproses
df

,Name,Image_url,Cuisine,Cook_time (in mins),Ingredients_name,Ingredients_quantity,Instructions
0,Khandeshi Dal Recipe,https://www.archanaskitchen.com/images/archana...,Maharashtrian Recipes,30.0,"Green Moong Dal (Whole), Pink Masoor Dal (Spli...",2 tablespoons Green Moong Dal (Whole) 2 table...,"To begin making the Khandeshi Dal recipe, we w..."
1,Padipe Saasmi Recipe - Udupi Style Amaranth Le...,https://www.archanaskitchen.com/images/archana...,Udupi,20.0,"Red Amaranth Leaves, Fresh coconut, Dry Red Ch...","12 sprig Red Amaranth Leaves , You can use spi...","To begin making the Padipe Saasmi Recipe, pluc..."
2,Creamy Polenta Recipe - Savory Cornmeal Porridge,https://www.archanaskitchen.com/images/archana...,Italian Recipes,20.0,"Polenta, Water, Milk, Extra Virgin Olive Oil, ...",1 cup Polenta 4 cups Water 1 cup Milk 1 tab...,"To begin making the Creamy Polenta Recipe, hea..."
3,Summer Lettuce Salad Recipe,https://www.archanaskitchen.com/images/archana...,Continental,0.0,"Cherry tomatoes, Red onion, Cucumber, Sweet co...","10 Cherry tomatoes , halved 1 Red onion , ong...","To begin making Summer Lettuce Salad Recipe, c..."
4,Dondakkai Puli Curry Recipe - South Indian Sty...,https://www.archanaskitchen.com/images/archana...,Tamil Nadu,25.0,"Tindora (Dondakaya/ Kovakkai), Black Urad Dal ...",150 grams Tindora (Dondakaya/ Kovakkai) 3 tab...,To begin making the Dondakkai Puli Curry Recip...
...,...,...,...,...,...,...,...
6874,Tuna Roasted Mushroom Sandwich Recipe with Tar...,https://www.archanaskitchen.com/images/archana...,Continental,30.0,"Ciabatta bread, Canned tuna, Button mushrooms,...","2 Ciabatta bread , halved 125 grams Canned tu...",To begin making Tuna Mushroom Tartare Dip Mayo...
6875,Tomato Onion Bruschetta With Mint Mayo Recipe,https://www.archanaskitchen.com/images/archana...,Italian Recipes,20.0,"Baguette, Onions, Tomatoes, Del Monte Mint May...","1 Baguette , sliced 2 Onions , finely chopped...",To begin making the Tomato Onion Bruschetta wi...
6876,Tomatoes Stuffed With Mint Mayo And Paneer Recipe,https://www.archanaskitchen.com/images/archana...,Continental,30.0,"Paneer (Homemade Cottage Cheese), Del Monte Mi...","1/4 cup Paneer (Homemade Cottage Cheese) , cru...",To begin making the Tomatoes Stuffed With Mint...
6877,Shrimp Stir Fry With Cheesy Dip Mayo Recipe,https://www.archanaskitchen.com/images/archana...,Asian,45.0,"Green Bell Pepper (Capsicum), Red Bell pepper ...","1/2 Green Bell Pepper (Capsicum) , julienned ...",To prepare Shrimp Stir Fry With Cheesy Dip May...


In [187]:
# Step 1: Gathering, Preparing, and Processing Data
def load_and_process_data(data_path):
    df = pd.read_csv(data_path)
    df['Ingredients_name'] = df['Ingredients_name'].astype(str)

    def tokenize_ingredients(ingredients):
        ingredients = re.sub(r'[^\w\s]', '', ingredients)  # Menghapus tanda baca
        tokens = ingredients.lower().split()  # Mengonversi ke huruf kecil dan membagi
        return tokens

    df['tokenized_ingredients'] = df['Ingredients_name'].apply(tokenize_ingredients)
    return df

In [188]:
# Step 2: Building, Training, and Deploying Model
def train_word2vec(df):
    model = Word2Vec(sentences=df['tokenized_ingredients'], vector_size=100, window=5, min_count=1, workers=4)
    return model

def get_recipe_vector(ingredients, model):
    vector = np.zeros(model.vector_size)
    count = 0
    for ingredient in ingredients:
        if ingredient in model.wv:
            vector += model.wv[ingredient]
            count += 1
    if count > 0:
        vector /= count
    return vector

def build_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(input_dim,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(input_dim, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model

def train_similarity_model(recipe_vectors):
    input_dim = recipe_vectors.shape[1]
    similarity_model = build_model(input_dim)

    # Split data into training and validation sets
    X_train, X_val = train_test_split(recipe_vectors, test_size=0.2, random_state=42)

    # Callback to display metrics
    class DisplayCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            print(f"Epoch {epoch + 1}: loss = {logs['loss']:.4f}, accuracy = {logs['accuracy']:.4f}, val_loss = {logs['val_loss']:.4f}, val_accuracy = {logs['val_accuracy']:.4f}")

    # Train the model
    similarity_model.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_val, X_val), callbacks=[DisplayCallback()])

    return similarity_model

In [189]:
# Step 3: Evaluating, Optimizing, and Deploying the Model
def recommend_recipes(user_allergy_ingredients, user_preference_ingredients, df, model, similarity_model, top_n=10):
    user_preference_vector = get_recipe_vector(user_preference_ingredients, model)
    user_preference_vector = similarity_model.predict(np.array([user_preference_vector]))[0]

    def contains_allergens(ingredients, allergens):
        return any(allergen in ingredients for allergen in allergens)

    # Filter out recipes that contain any of the allergy ingredients
    filtered_df = df[~df['tokenized_ingredients'].apply(lambda x: contains_allergens(x, user_allergy_ingredients))]

    # If no recipes are available after filtering, return an empty list
    if filtered_df.empty:
        return []

    similarities = cosine_similarity([user_preference_vector], filtered_df['recipe_vector'].tolist())[0]
    sorted_indices = np.argsort(similarities)[::-1]
    top_indices = sorted_indices[:top_n]

    recommendations = filtered_df.iloc[top_indices]

    result = [filtered_df.index[top_indices[idx]] for idx in range(len(recommendations))]

    return result
# Function to train the model
def train_model(data_path):
    # Step 1
    df = load_and_process_data(data_path)

    # Step 2
    word2vec_model = train_word2vec(df)

    # Generate recipe vectors
    df['recipe_vector'] = df['tokenized_ingredients'].apply(lambda x: get_recipe_vector(x, word2vec_model))
    recipe_vectors = np.stack(df['recipe_vector'].values)

    # Train similarity model
    similarity_model = train_similarity_model(recipe_vectors)

    # Save the model
    similarity_model.save('/content/drive/MyDrive/dataset/recipe_model.h5')

    return df, word2vec_model, similarity_model

# Run the training process
data_path = '/content/drive/MyDrive/dataset/Food_Recipes_Cleaned.csv'
df, word2vec_model, similarity_model = train_model(data_path)


Epoch 1/10
172/172 [==============================] - 1s 4ms/step - loss: 0.0230 - accuracy: 0.4523 - val_loss: 0.0049 - val_accuracy: 0.5901
Epoch 2/10
172/172 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 0.6702 - val_loss: 0.0018 - val_accuracy: 0.6882
Epoch 3/10
172/172 [==============================] - 1s 3ms/step - loss: 0.0013 - accuracy: 0.7214 - val_loss: 0.0011 - val_accuracy: 0.7129
Epoch 4/10
172/172 [==============================] - 1s 3ms/step - loss: 8.7414e-04 - accuracy: 0.7369 - val_loss: 8.3650e-04 - val_accuracy: 0.7413
Epoch 5/10
172/172 [==============================] - 1s 3ms/step - loss: 6.8763e-04 - accuracy: 0.7481 - val_loss: 6.6991e-04 - val_accuracy: 0.7493
Epoch 6/10
172/172 [==============================] - 1s 4ms/step - loss: 5.6363e-04 - accuracy: 0.7503 - val_loss: 5.9940e-04 - val_accuracy: 0.7551
Epoch 7/10
172/172 [==============================] - 1s 5ms/step - loss: 4.9618e-04 - accuracy: 0.7552 - val_loss: 5.2883e-

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [190]:
# Step 4: Finalize Training Model and Deployment
def run_recommendation_system(df, word2vec_model, similarity_model, top_n=30):
    user_allergy_ingredients = input("Enter allergy ingredients (comma separated): ").lower().split(',')
    user_preference_ingredients = input("Enter preferred ingredients (comma separated): ").lower().split(',')

    recommendations = recommend_recipes(user_allergy_ingredients, user_preference_ingredients, df, word2vec_model, similarity_model, top_n)
    return recommendations

# Run the recommendation system with the trained model
recommendations = run_recommendation_system(df, word2vec_model, similarity_model, top_n=30)
print(recommendations)

Enter allergy ingredients (comma separated): egg, sauce, milk
Enter preferred ingredients (comma separated): chicken, cheese, rice


1/1 [==============================] - 0s 79ms/step
[3572, 4044, 1727, 6445, 4929, 2068, 2976, 386, 1698, 735, 6313, 1237, 406, 2128, 1026, 2690, 1003, 2631, 380, 3537, 4500, 3048, 3707, 5462, 3927, 4907, 731, 3571, 4109, 1077]
